<a href="https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutoriel : Application de BERT via DistillBERT

Ce tutoriel est une traduction de celui proposé par Jay Alammar.  
Son blog : https://jalammar.github.io/  
L'article de son blog pour lequel il a créé le notebook :  https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/  
Une traduction française de cet article :  

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" />

Dans ce notebook, nous allons utiliser un modèle de deep learning pré-entrainé pour traiter certains textes. Nous utiliserons ensuite les résultats de ce modèle pour en effectuer une classification. Le texte étant une liste de phrases tirées de critiques de films. Nous nous limiterons à une classification bianire où les phrases seront classées soient comme "positives" soit comme "négative".



## Classification de sentiments

Notre but est de créer un modèle qui prend une phrase (tout comme celles de notre ensemble de données) et produit soit 1 (indiquant que la phrase porte un sentiment positif) ou 0 (indiquant que la phrase porte un sentiment négatif). On peut penser que ça ressemble à ça :

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Sous le capot, le modèle est en fait composé de deux modèles :

* DistilBERT qui traite la phrase et transmet les informations qu’il en extrait au modèle suivant. DistilBERT est une version plus petite de BERT développée et open source par l’équipe de HuggingFace. C’est une version plus légère et plus rapide de BERT (40% plus léger et 60% plus rapide) et ayant des performances semblables (à 97%).
* Le modèle suivant, une basique régression de scikit learn, qui prend en compte le résultat du traitement de DistilBERT et classe la phrase comme positive ou négative (1 ou 0, respectivement). =

Les données que nous passons entre les deux modèles sont un vecteur de taille 768. On peut imaginer ce vecteur comme un embedding de la phrase que l’on peut utiliser pour la classification.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />



## Dataset

Le jeu de données que nous utiliserons dans cet exemple est [SST2](https://nlp.stanford.edu/sentiment/index.html), qui contient des phrases de critiques de films, chacune étant labélisée positive (a la valeur 1) ou négative (a la valeur 0):


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>


## Installation de la librairie transformers 
Commençons par installer la bibliothèque de transformateurs huggingface pour que nous puissions charger notre modèle NLP d'apprentissage profond.

In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 53.8MB/s 
     |████████████████████████████████| 645kB 39.9MB/s 
     |████████████████████████████████| 860kB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=cb6e175a99d8f14f69f593b734ea73303c23a7dbdc694be65435cfeebd1f3124
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importation du dataset
Utilisons pandas pour lire le jeu de données et le charger dans un dataframe.

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

Pour des raisons de performance, nous n'utiliserons que 2 000 phrases de l'ensemble du jeu de données.

In [0]:
batch_1 = df[:2000]

On peut demander à pandas combien de phrases sont labélisées "positives" (valeur 1) et combien sont labélisées "négatives" (valeur 0).

In [0]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Charger le modèle BERT pré-entrainé 

In [0]:
# Pour DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Vous souhaitez utiliser BERT au lieu de distilBERT? 
## Décommenter la ligne suivante:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Chargement du mdoele pré-entrainé et du tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

100%|██████████| 267967963/267967963 [00:03<00:00, 72728701.55B/s]


Actuellement, la variable `model` contient un modèle distilBERT pré-entraîné : une version de BERT qui est plus petite, mais beaucoup plus rapide et nécessite beaucoup moins de mémoire.


## Modèle #1 : Préparation du dataset
Avant de pouvoir donner nos phrases à BERT, nous avons besoin d'un traitement minimal pour les mettre dans le format requis.

#### Tokenisation
Notre première étape consiste à tokenizer les phrases , c'est à dire les décomposer en mots et en sous-mots dans le format avec lequel BERT est à l'aise.

In [0]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding (rembourrage)

Après la tokenization, `tokenized` est une liste de phrases : chaque phrase est représentée comme une liste de tokens. Nous voulons que BERT traite nos exemples en une seule fois (en un seul lot). C'est plus rapide comme ça. Pour cette raison, nous devons remplir toutes les listes à la même taille, de sorte que nous puissions représenter l'entrée comme un tableau à deux dimensions, plutôt que comme une liste de listes (de différentes longueurs).

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Notre jeu de données est maintenant dans la variable `padded`, nous pouvons voir ses dimensions ci-dessous :

In [0]:
np.array(padded).shape

(2000, 59)

### Masking
Si nous envoyons directement `padded` à BERT, ce serait un peu confus. Nous devons créer une autre variable pour lui dire d'ignorer (masquer) le rembourrage que nous avons ajouté quand il traite son entrée. Voilà ce qu'est un masque d'attention :

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

## Model #1: Et maintenant place au Deep Learning!
Maintenant que notre modèle et nos données sont prêts, lançons notre modèle !

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

La fonction `model()` fait passer nos phrases par BERT. Les résultats du traitement seront retournés dans `last_hidden_states`.

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Coupons seulement la partie de la sortie dont nous avons besoin. C'est le résultat correspondant au premier token de chaque phrase.  
La façon dont BERT procède à la classification des phrases est d'ajouter un token appelé " [CLS] " (pour classification) au début de chaque phrase. La sortie correspondant à ce token peut être considérée comme un embedding pour toute la phrase.



<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

Nous les enregistrerons dans la variable `features`, car elles serviront de caractéristiques à notre modèle de régression logitique.

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

Les labels indiquant quelle phrase est positive et négative sont assignés dans la variable `labels`.

In [0]:
labels = batch_1[1]

## Modele #2: Découpage Entraînement/Test

Séparons maintenant notre jeu de données en un jeu d'entraînement et un jeu de test (même si nous utilisons 2 000 phrases du jeu d'apprentissage SST2).

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />

### [Bonus] Grille de recherche des paramètres optimaux
Nous pouvons plonger dans la Régression Logistique directement avec les paramètres par défaut de Scikit Learn. Mais parfois cela vaut la peine de chercher la meilleure valeur du paramètre C, qui détermine la force de régularisation.

In [0]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

Nous entraînons maintenant le modèle LogisticRegression(). Si vous avez choisi d'opter pour la grille de recherche, vous pouvez insérer la valeur de C dans la déclaration du modèle (par exemple `LogisticRegression(C=5.2)`).

In [0]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluation du Model #2

Dans quelle mesure notre modèle permet-il de classer les phrases ? L'un des moyens consiste à vérifier la précision par rapport à l'ensemble de données test :

In [0]:
lr_clf.score(test_features, test_labels)

0.824

Est-cet un bon score ? A quoi peut-on le comparer ? Regardons d'abord un classificateur factice :

In [0]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.527 (+/- 0.05)


Notre modèle est donc nettement plus performant qu'un classificateur factice. Mais comment se compare-t-il aux meilleurs modèles ?


## Benchmark des scores SST2 
Le [score obtenant actuellement le meilleur taux](http://nlpprogress.com/english/sentiment_analysis.html) pour ce dataset est de **96.8%**. DistilBERT peut être entrainé pour améliorer le score pour cette tache de classification  – un processus appellé **fine-tuning**. Cela met à jour les poids de BERT pour permettre d'obtenir une meilleure performance. Le DistilBERT "fine-tuné" permet d'obtenir un score de **90.7%**. Le BERT Large permet d'obtenir quant à lui un score de **94.9%**.


Et c'est tout ! C'est un bon premier contact avec BERT.  
L'étape suivante serait de consulter la documentation et d'essayer le [fine-tuning] (https://huggingface.co/transformers/examples.html#glue). Vous pouvez également revenir en arrière et passer de distilBERT à BERT et voir comment cela fonctionne.